In [27]:

#PREDICTING TRANSLATION INITIATION SITES FROM GENOMIC SEQUENCES



In [28]:
import pandas as pd
import numpy as np
import itertools
import pandas as pd
from sklearn.ensemble import RandomForestClassifier


In [29]:
data1 = pd.read_csv("mmc1.csv")

In [30]:
data1.head()

,Sequence,Label
0,GATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGCTGGGAA...,True
1,GTCACTGCCCTCGCGGCCAATCCAAGGGCTGCGCGCGTGTCCCTTA...,True
2,GTCACTGTCCTCGGGGCCAATCCAAGGGCTGCGCGCGTGTCCCTTA...,True
3,GCGGAAAACGGCAGGAGGAGAGCCAATCCCGAGGGTCGGCGGACGC...,True
4,TCGGCGGTGGAACCGCCAGTCCGGGGTCACAGAGCTTGAGAAGCGA...,True


In [31]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2318 entries, 0 to 2317
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sequence  2318 non-null   object
 1   Label     2318 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 20.5+ KB


In [32]:
# check len of sequence in each row

def len_seq(sequence):
    return len(sequence)
data1['len'] = data1['Sequence'].apply(len_seq)

# remove gaps from sequence

def remove_gaps(sequence):
    return sequence.replace(' ','')
data1['Sequence'] = data1['Sequence'].apply(remove_gaps)

In [33]:
data1.head()

,Sequence,Label,len
0,GATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGCTGGGAA...,True,399
1,GTCACTGCCCTCGCGGCCAATCCAAGGGCTGCGCGCGTGTCCCTTA...,True,399
2,GTCACTGTCCTCGGGGCCAATCCAAGGGCTGCGCGCGTGTCCCTTA...,True,399
3,GCGGAAAACGGCAGGAGGAGAGCCAATCCCGAGGGTCGGCGGACGC...,True,399
4,TCGGCGGTGGAACCGCCAGTCCGGGGTCACAGAGCTTGAGAAGCGA...,True,399


In [34]:
# see unique values in len column
data1['len'].unique()

array([399], dtype=int64)

In [35]:
# check if there are null values in the dataset
data1.isnull().sum()


Sequence    0
Label       0
len         0
dtype: int64

In [36]:
# removing null values
data1.dropna(inplace=True)

In [37]:
# remove duplicates from the dataset
data1["Sequence"].drop_duplicates(inplace=True)

In [38]:
# check if there are any duplicates in the sequence column
data1['Sequence'].duplicated().sum()


1

In [39]:
# remove the sequence if there are any other nucleotides other than A,T,C,G

def check_seq(sequence):
    for i in sequence:
        if i not in ['A','T','C','G']:
            return False
    return True

data1["h"] = data1['Sequence'].apply(check_seq)

# remove the sequence if there are any other nucleotides other than A,T,C,G
data1 = data1[data1['Sequence'].apply(check_seq)]
data1[data1['h']==False]
data1.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2318 entries, 0 to 2317
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sequence  2318 non-null   object
 1   Label     2318 non-null   bool  
 2   len       2318 non-null   int64 
 3   h         2318 non-null   bool  
dtypes: bool(2), int64(1), object(1)
memory usage: 58.9+ KB


In [40]:
# C2 encoding of sequence
def C2_encoding(sequence):
    encoding = []
    for i in sequence:
        if i == 'A':
            encoding.append([0,0])
        elif i == 'C':
            encoding.append([0,1])
        elif i == 'G':
            encoding.append([1,0])
        elif i == 'T':
            encoding.append([1,1])
    return encoding


In [41]:
data1['C2_encoding'] = data1['Sequence'].apply(C2_encoding)

In [42]:
data1.head()

,Sequence,Label,len,h,C2_encoding
0,GATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGCTGGGAA...,True,399,True,"[[1, 0], [0, 0], [1, 1], [0, 1], [0, 1], [0, 1..."
1,GTCACTGCCCTCGCGGCCAATCCAAGGGCTGCGCGCGTGTCCCTTA...,True,399,True,"[[1, 0], [1, 1], [0, 1], [0, 0], [0, 1], [1, 1..."
2,GTCACTGTCCTCGGGGCCAATCCAAGGGCTGCGCGCGTGTCCCTTA...,True,399,True,"[[1, 0], [1, 1], [0, 1], [0, 0], [0, 1], [1, 1..."
3,GCGGAAAACGGCAGGAGGAGAGCCAATCCCGAGGGTCGGCGGACGC...,True,399,True,"[[1, 0], [0, 1], [1, 0], [1, 0], [0, 0], [0, 0..."
4,TCGGCGGTGGAACCGCCAGTCCGGGGTCACAGAGCTTGAGAAGCGA...,True,399,True,"[[1, 1], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0..."


In [43]:
data1["Label"].value_counts()


True     1159
False    1159
Name: Label, dtype: int64

In [44]:
# gapped kmer in the sequence
import itertools
def gapped_kmer(sequence, k=6, g=2):
    encoding = []
    freq_dict = {}
    for i in range(len(sequence) - k + 1):
        encoding.append(sequence[i:i+k])
    return encoding, freq_dict

def get_kmer_frequencies(seq,k):
    freq_dict = {}
    l=['A','T','C','G']
    repetitions = k
    combinations = list(itertools.product(l, repeat=repetitions))
    for i in combinations:
        se=''.join(list(i))
        fo=seq.count(se)
        freq_dict[se]=fo
    total_kmers = len(seq) - k + 1
    for kmer in freq_dict:
        freq_dict[kmer] /= total_kmers
    return freq_dict

    
                


data1['gapped_kmer'] = data1['Sequence'].apply(gapped_kmer)
data1["kmer_frequencies_tri"] = data1["Sequence"].apply(get_kmer_frequencies, k=3)
data1["kmer_frequencies_di"] = data1["Sequence"].apply(get_kmer_frequencies, k=2)
data1["C2_encoding"]

0       [[1, 0], [0, 0], [1, 1], [0, 1], [0, 1], [0, 1...
1       [[1, 0], [1, 1], [0, 1], [0, 0], [0, 1], [1, 1...
2       [[1, 0], [1, 1], [0, 1], [0, 0], [0, 1], [1, 1...
3       [[1, 0], [0, 1], [1, 0], [1, 0], [0, 0], [0, 0...
4       [[1, 1], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0...
                              ...                        
2313    [[0, 0], [1, 0], [1, 0], [0, 0], [1, 0], [0, 0...
2314    [[1, 0], [1, 1], [1, 0], [1, 0], [0, 0], [0, 0...
2315    [[0, 0], [0, 1], [0, 0], [1, 0], [0, 0], [0, 0...
2316    [[0, 0], [0, 0], [1, 1], [0, 1], [1, 1], [1, 0...
2317    [[0, 0], [0, 1], [1, 1], [0, 1], [1, 1], [1, 1...
Name: C2_encoding, Length: 2318, dtype: object

In [45]:
def atg_freq(sequence):
    total_atg = sequence.count('ATG')
    
        
    return total_atg
def atg_features(sequence):
    inframe_atg = 0
    for i in range(len(sequence)):
        if i % 3 == 0 and sequence[i:i+3] == 'ATG': 
            inframe_atg += 1
    return inframe_atg


data1['atg_freq'] = data1['Sequence'].apply(atg_freq)
data1["atg_features"]=data1['Sequence'].apply(atg_features)

In [46]:

# split kmer_frequencies keys as columns
data1 = pd.concat([data1, data1['kmer_frequencies_di'].apply(pd.Series)], axis=1)
data1 = pd.concat([data1, data1['kmer_frequencies_tri'].apply(pd.Series)], axis=1)
data1.drop('kmer_frequencies_tri', axis=1, inplace=True)
data1.drop('kmer_frequencies_di', axis=1, inplace=True)
# replace nan  with 0
data1.fillna(0, inplace=True)
data1.head()


,Sequence,Label,len,h,C2_encoding,gapped_kmer,atg_freq,atg_features,AA,AT,...,GTC,GTG,GCA,GCT,GCC,GCG,GGA,GGT,GGC,GGG
0,GATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGCTGGGAA...,True,399,True,"[[1, 0], [0, 0], [1, 1], [0, 1], [0, 1], [0, 1...","([GATCCC, ATCCCT, TCCCTG, CCCTGC, CCTGCG, CTGC...",2,1,0.022613,0.025126,...,0.017632,0.022670,0.010076,0.027708,0.022670,0.027708,0.027708,0.035264,0.020151,0.052897
1,GTCACTGCCCTCGCGGCCAATCCAAGGGCTGCGCGCGTGTCCCTTA...,True,399,True,"[[1, 0], [1, 1], [0, 1], [0, 0], [0, 1], [1, 1...","([GTCACT, TCACTG, CACTGC, ACTGCC, CTGCCC, TGCC...",2,1,0.027638,0.010050,...,0.017632,0.022670,0.015113,0.015113,0.052897,0.062972,0.022670,0.020151,0.060453,0.030227
2,GTCACTGTCCTCGGGGCCAATCCAAGGGCTGCGCGCGTGTCCCTTA...,True,399,True,"[[1, 0], [1, 1], [0, 1], [0, 0], [0, 1], [1, 1...","([GTCACT, TCACTG, CACTGT, ACTGTC, CTGTCC, TGTC...",2,1,0.027638,0.012563,...,0.015113,0.025189,0.015113,0.017632,0.045340,0.068010,0.022670,0.020151,0.062972,0.035264
3,GCGGAAAACGGCAGGAGGAGAGCCAATCCCGAGGGTCGGCGGACGC...,True,399,True,"[[1, 0], [0, 1], [1, 0], [1, 0], [0, 0], [0, 0...","([GCGGAA, CGGAAA, GGAAAA, GAAAAC, AAAACG, AAAC...",1,1,0.025126,0.010050,...,0.020151,0.020151,0.015113,0.010076,0.040302,0.078086,0.030227,0.035264,0.080605,0.045340
4,TCGGCGGTGGAACCGCCAGTCCGGGGTCACAGAGCTTGAGAAGCGA...,True,399,True,"[[1, 1], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0...","([TCGGCG, CGGCGG, GGCGGT, GCGGTG, CGGTGG, GGTG...",1,1,0.015075,0.012563,...,0.017632,0.010076,0.012594,0.017632,0.068010,0.025189,0.015113,0.012594,0.017632,0.015113


In [47]:
# check the len of gapped_kmer
data1['gapped_kmer'].apply(len).unique()
# label encoding of the label column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data1['Label'] = le.fit_transform(data1['Label'])
data1.head()
data1.shape


(2318, 88)

In [48]:
# pca on di kmer and tri kmer 
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
features = pca.fit_transform(data1.iloc[:, 7:])

data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2318 entries, 0 to 2317
Data columns (total 88 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sequence      2318 non-null   object 
 1   Label         2318 non-null   int64  
 2   len           2318 non-null   int64  
 3   h             2318 non-null   bool   
 4   C2_encoding   2318 non-null   object 
 5   gapped_kmer   2318 non-null   object 
 6   atg_freq      2318 non-null   int64  
 7   atg_features  2318 non-null   int64  
 8   AA            2318 non-null   float64
 9   AT            2318 non-null   float64
 10  AC            2318 non-null   float64
 11  AG            2318 non-null   float64
 12  TA            2318 non-null   float64
 13  TT            2318 non-null   float64
 14  TC            2318 non-null   float64
 15  TG            2318 non-null   float64
 16  CA            2318 non-null   float64
 17  CT            2318 non-null   float64
 18  CC            2318 non-null 

In [49]:
features

array([[-1.79398635e+00,  5.60599667e-02, -2.09631436e-02, ...,
        -6.43481298e-04, -8.98822008e-03,  1.26233456e-02],
       [-1.79708680e+00,  1.50122721e-01,  6.07577946e-03, ...,
         2.89483555e-03, -3.07385053e-03, -6.73447432e-03],
       [-1.79705853e+00,  1.49481306e-01, -8.01013651e-04, ...,
         2.01745074e-03,  4.41346648e-04, -6.92782148e-03],
       ...,
       [ 1.21152343e+00, -5.03814193e-02, -5.01503757e-02, ...,
         3.86686148e-03, -5.57898818e-05, -5.69071658e-03],
       [ 2.11647335e-01, -7.31042573e-02, -6.32479290e-02, ...,
        -3.56108402e-03, -1.83378867e-04, -5.69868236e-03],
       [-7.87806377e-01, -1.06851846e-01, -6.41792714e-02, ...,
        -4.33084018e-03,  2.99074502e-05, -6.69111437e-03]])

In [50]:
import csv

with open('mmc1.csv', 'r') as file:
    reader = csv.DictReader(file)
    h=[]
    for row in reader:
        if row['Label'] == 'True':
            h.append(1)
        else:
            h.append(0)


In [56]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(features, h, test_size=0.3)
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))





Accuracy: 0.9037356321839081


In [57]:
def predicting(a):
    l=[]
    l.append(atg_freq(a))
    l.append(atg_features(a))
    f2=list(get_kmer_frequencies(a,k=2).values())
    f3=list(get_kmer_frequencies(a,k=3).values())
    final=l+f2+f3
    final=np.array(final)
    my_dataframe=data1.iloc[:, 6:]
    new_dataframe = pd.DataFrame([final], columns=my_dataframe.columns)
    my_dataframe= pd.concat([my_dataframe, new_dataframe], ignore_index=True)
    
    pca = PCA(n_components=20)
    last= pca.fit_transform(my_dataframe)
    return last[-1]

a="AAGGAGAGGCCCGGACTTGGGCATATCTGCAGAAAAACCCTTCCCCACTAGGCAGGCGCGGGGGAGGGCGTGGAGGGGCGGGGTGGTGCCGCCCCCGGGGCGGGCCCAGTGCGTGGCAGCGGGACCTGCGGCCCCGTCGCGAAGTTTCCAGCCCTGCGAGCGCCGCCGGGTCGGCCGATCGTCCCCCATACCTCGGCCATGCGGCCCCTGCTGCTACTGGCCCTGCTGGGCTGGCTGCTGCTGGCCGAAGCGAAGGGCGACGCCAAGCCGGAGGGTGAGGGAGCGAAGGCCGGGGGCGGGAGCGCGGATCCGGGCGGGAGGGCTGGTGTCGGGCTGCCTCCCTGGGAACGACCTGAATGGGAGGCCTGGGCTGGAGAGGGAGTCTGGGTTCCGGCTCCT"
b="GAAATATCACTTTGTAGATTGTACAAAAAGACTGTTTCTAAACTGCTCAATCAAAAGAAAATTTCAAACTGGTGAGATGAATGCACACATAACAAAGGAGTTTCTCAGGAATCTTTTTTTTTTATTTTCAACACTTTTTTATTTCTTTCAAAGTTAGTTTTTTAATTTATTATTATTATATTTTAAGTTTTAGGGTACATGTGCACAATGTGCAGGTTTGTTACATATGTATACATGTGCCATGCTGGTGTGCTGCACCCACTAACTCGTCATCTAGCATTAGGTATATCTCCCAATGCTATCCCTCCCCCTTCCCCCCACCCCACAACAGTCCCCAGAGTGTGATATTCCCCTTCCTGTGTCCATATGTTGCTCTCATTGTTCAATTCCCACCCATGA"


tis=[]
tis.append(predicting(a))
tis.append(predicting(b))
y_pred = clf.predict(tis)
y_pred


array([1, 0])

In [62]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, h, test_size=0.3)

rfc = RandomForestClassifier(n_estimators=500)

# Train the classifier on the training data
rfc.fit(X_train, y_train)

# Test the classifier on the testing data
accuracy = rfc.score(X_test, y_test)
print("Accuracy:", accuracy)


Accuracy: 0.9971264367816092


In [63]:
a="AAGGAGAGGCCCGGACTTGGGCATATCTGCAGAAAAACCCTTCCCCACTAGGCAGGCGCGGGGGAGGGCGTGGAGGGGCGGGGTGGTGCCGCCCCCGGGGCGGGCCCAGTGCGTGGCAGCGGGACCTGCGGCCCCGTCGCGAAGTTTCCAGCCCTGCGAGCGCCGCCGGGTCGGCCGATCGTCCCCCATACCTCGGCCATGCGGCCCCTGCTGCTACTGGCCCTGCTGGGCTGGCTGCTGCTGGCCGAAGCGAAGGGCGACGCCAAGCCGGAGGGTGAGGGAGCGAAGGCCGGGGGCGGGAGCGCGGATCCGGGCGGGAGGGCTGGTGTCGGGCTGCCTCCCTGGGAACGACCTGAATGGGAGGCCTGGGCTGGAGAGGGAGTCTGGGTTCCGGCTCCT"
b="GAAATATCACTTTGTAGATTGTACAAAAAGACTGTTTCTAAACTGCTCAATCAAAAGAAAATTTCAAACTGGTGAGATGAATGCACACATAACAAAGGAGTTTCTCAGGAATCTTTTTTTTTTATTTTCAACACTTTTTTATTTCTTTCAAAGTTAGTTTTTTAATTTATTATTATTATATTTTAAGTTTTAGGGTACATGTGCACAATGTGCAGGTTTGTTACATATGTATACATGTGCCATGCTGGTGTGCTGCACCCACTAACTCGTCATCTAGCATTAGGTATATCTCCCAATGCTATCCCTCCCCCTTCCCCCCACCCCACAACAGTCCCCAGAGTGTGATATTCCCCTTCCTGTGTCCATATGTTGCTCTCATTGTTCAATTCCCACCCATGA"


tis=[]
tis.append(predicting(a))
tis.append(predicting(b))
predict=rfc.predict(tis)
predict

array([1, 0])